# Image Segmentation - Thresholding 

In [2]:
# Import necessary libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

ModuleNotFoundError: No module named 'cv2'